In [2]:
import json
import os
import re
import MeCab
import numpy as np
import keras
from keras.utils import np_utils
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.preprocessing import sequence
import collections

# import gensim.parsing.preprocessing
from gensim import corpora, matutils
from sklearn.model_selection import train_test_split

DIC_NAME = 'dic_raw_full4.txt'

def load_json(data_dir):
    with open(os.path.join(data_dir, 'livedoor.json')) as f:
        items = json.load(f)
    return items

def clean_text(text):
    replaced_text = '\n'.join(s.strip() for s in text.splitlines()[2:] if s != '')  # skip header by [2:]
    replaced_text = replaced_text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text

def tokenize(text):
    mecabTagger = MeCab.Tagger('mecal-ipadic-neologd')
    word_list = []
    res = mecabTagger.parseToNode(text)
    while res:
        pos = res.feature.split(",")
        if pos[0] in ["名詞"]:
            if not pos[1] in ["代名詞", "固有名詞", "数", "非自立", "特殊"]:
                try:
                    word_list.append(res.surface)
                except UnicodeDecodeError:
                    print('デコードエラー→'+pos[0]+pos[1]+pos[2])
        res = res.next
    return word_list

def make_words_list(data):
    words_list = [clean_text(text) for text in data]
    words_list = [tokenize(text) for text in words_list]
    return words_list

def load_dic(project_dir, data, words_list):
    DIC_DIR = os.path.join(project_dir, 'dic', DIC_NAME)
    if not os.path.exists(DIC_DIR):
        dictionary = corpora.Dictionary(words_list)
        dictionary.filter_extremes(no_below=2, no_above=0.8)
        dictionary.save_as_text(DIC_DIR)
    dic = corpora.Dictionary.load_from_text(DIC_DIR)
    return dic

def make_data_set(words_list, dic):
    # 辞書の次元→ len(dic.keys()) or len(dic.values())
    vecs = [dic.doc2bow(word_list) for word_list in words_list]
    x = [matutils.corpus2dense([vec], num_terms=len(dic)).T[0] for vec in vecs]
    y = items['label']
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    y_train = np.array(y_train)
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return x_train, x_test, y_train, y_test, x

def make_model(input_dim, output_dim):
    # set parameters:
    first_hidden=400
    second_hidden=200
    third_hidden=100
    fourth_hidden=50

    # print('Build model...')
    model = Sequential()
    model.add(Dense(first_hidden, input_dim=input_dim,  activation="relu"))
    model.add(Dense(second_hidden, input_dim=first_hidden,  activation="relu"))
    model.add(Dense(third_hidden, input_dim=second_hidden,  activation="relu"))
    model.add(Dense(fourth_hidden, input_dim=third_hidden,  activation="relu"))
    model.add(Dense(output_dim, input_dim=fourth_hidden,  activation="softmax"))
    return model

def load_model(input_dim, output_dim):
    model_path = os.path.join(project_dir, 'model/model_json1.json')
    if not os.path.exists(model_path):
        print('made {0}'.format(model_path))
        model = make_model(input_dim, output_dim)
        with open(model_path, 'w') as f:
            json.dump(model.to_json(), f)
    else:
        with open(model_path, 'r') as f:
            try:
                json_string = json.load(f)
                model = model_from_json(json_string)
                print('loaded from {0}'.format(model_path))
            except UnicodeDecodeError:
                model = make_model(input_dim, output_dim)
                print('made {0}/{1}.'.format(model_path.split('/')[-2], model_path.split('/')[-1]))
    return model, model_path

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
project_dir = os.getcwd()
DATA_DIR = os.path.join(project_dir, 'data/processed')
items = load_json(DATA_DIR)
words_list = make_words_list(items['data'])
dic = load_dic(project_dir, items['data'], words_list)

http://news.livedoor.com/article/detail/4778030/
2010-05-22T14:30:00+0900
友人代表のスピーチ、独女はどうこなしている？
　もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？　さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない。

　「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」

　さてそんなとき、独女はどう対応したらいいか？

　最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。

　「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。

　しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）

　「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」
　
　サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ。しかしそれも上手に対応できる人なら

In [112]:
x_train, x_test, y_train, y_test, x = make_data_set(words_list, dic)
model, model_path = load_model(input_dim=len(x_train[0]), output_dim=len(y_train[0]))
model.summary()
earlystopping = keras.callbacks.EarlyStopping(monitor='acc', verbose=1, patience=5, mode='auto')
model_checkpoint = keras.callbacks.ModelCheckpoint(model_path, monitor='acc', save_best_only=True, mode='auto', period=1)
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=200, batch_size=128, callbacks=[earlystopping, model_checkpoint])

made model/model_json1.json.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 400)               8008800   
_________________________________________________________________
dense_27 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_28 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_29 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_30 (Dense)             (None, 9)                 459       
Total params: 8,114,609
Trainable params: 8,114,609
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
5893/5893 [==============================] - 21s 4ms/step - loss: 0.6873 - acc: 0.8013
Epoc

In [113]:
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
classes = model.predict_classes(x_test, batch_size=128)
proba = model.predict_proba(x_test, batch_size=128)

# 各ラベルの項目数をカウント
count_list = collections.Counter(classes)
for k in sorted(count_list.keys()):
    print("category {0} ({1}) has {2} items".format(k, items['label_names'][str(k)], count_list[k]))

# 各ラベルにとって典型的なデータをそれぞれ表示
counter1 = range(len(y_test[0]))
counter2 = range(len(x[0]))
typical_list = [(np.argmax(proba[:,j])) for j in range(len(y_test[0]))]
print(typical_list)
for index, count1 in zip(typical_list, counter1):
    print("Most typical content in category {0} ({1}) is this below".format(count1, items['label_names'][str(count1)]))
    for each_x, count2 in zip(x, counter2):
        if np.allclose(x_test[index], each_x):
            print(items['data'][count2])
            break

# 間違ったラベルへの分類をしたデータを確認
for i in range(len(y_test)):
    itemindex = np.where(y_test[i] == 1)
    if classes[i] != itemindex[0]:
        print("count {0}  ==>  wrong predict : {1} , answer is {2}".format(i, classes[i], itemindex[0][0]))

1474/1474 [==============================] - 2s 1ms/step
acc: 92.06%
category 0 (dokujo-tsushin) has 178 items
category 1 (it-life-hack) has 159 items
category 2 (kaden-channel) has 176 items
category 3 (livedoor-homme) has 105 items
category 4 (movie-enter) has 178 items
category 5 (peachy) has 155 items
category 6 (smax) has 179 items
category 7 (sports-watch) has 184 items
category 8 (topic-news) has 160 items
count 14  ==>  wrong predict : 3 , answer is 6
count 36  ==>  wrong predict : 5 , answer is 0
count 38  ==>  wrong predict : 0 , answer is 5
count 47  ==>  wrong predict : 3 , answer is 5
count 68  ==>  wrong predict : 4 , answer is 2
count 70  ==>  wrong predict : 0 , answer is 5
count 86  ==>  wrong predict : 5 , answer is 0
count 103  ==>  wrong predict : 8 , answer is 5
count 114  ==>  wrong predict : 2 , answer is 8
count 117  ==>  wrong predict : 2 , answer is 3
count 128  ==>  wrong predict : 7 , answer is 8
count 145  ==>  wrong predict : 5 , answer is 0
count 158  ==>